# Materials design by *ab initio* methods (MDAM)

Prof. Blazej Grabowski, Dr. Yuji Ikeda

Department of Materials Design, University of Stuttgart

**Please submit:**

- **Original Jupyter Notebooks (XXX.ipynb) of your answers**
  - **Before submission, please do "Kernel – Restart & Run All" to check if your notebook is executable.**
- **PDF of "Print Preview"**
- **Other files requested in the exercise**

# Homework 02: Schrödinger equation in 1D real space

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interactive, fixed, IntSlider

## Fourth-Order-ACCURACY (5 pt)

Consider the *Dirichlet boundary condition* again for a single particle in 1D.  
- Instead of the second-order-accuracy finite difference, create the kinetic-energy matrix with the **fourth**-order-accuracy finite difference method.
- Calculate the eigenvalues of the kinetic-energy matrix **with the fourth-order accuracy**. Compare the thus-obtained eigenvalues with the eigenvalues with the second-order accuracy.

**Do not confuse the second-order "derivative" with the second-order "accuracy".**

Ref. https://en.wikipedia.org/wiki/Finite_difference_coefficient#Central_finite_difference

The fourth-order approximation is proved as follows:

\begin{alignat*}{99}
f(x + 2h) &= f(x) {}+{} &2hf'(x)& {}+{} & \frac{(+2h)^2}{2!}f''(x)& {}+{} & \frac{(+2h)^3}{3!}f'''(x)& {}+{} & \frac{(+2h)^4}{4!}f^{(4)}(x)& {}+{} & \frac{(+2h)^4}{5!}f^{(5)}(x)& {}+{} \mathcal{O}(h^6)& \quad(1) \\
f(x +  h) &= f(x) {}+{} & hf'(x)& {}+{} & \frac{(+ h)^2}{2!}f''(x)& {}+{} & \frac{(+ h)^3}{3!}f'''(x)& {}+{} & \frac{(+ h)^4}{4!}f^{(4)}(x)& {}+{} & \frac{(+ h)^4}{5!}f^{(5)}(x)& {}+{} \mathcal{O}(h^6)& \quad(2) \\
f(x)      &= f(x) {} {} &       & {} {} & \phantom{\frac{1}{1}}   & {} {} &                          & {} {} &                             & {} {} &                             & {} {}                 & \quad(3) \\
f(x -  h) &= f(x) {}-{} & hf'(x)& {}+{} & \frac{(- h)^2}{2!}f''(x)& {}+{} & \frac{(- h)^3}{3!}f'''(x)& {}+{} & \frac{(- h)^4}{4!}f^{(4)}(x)& {}+{} & \frac{(- h)^4}{5!}f^{(5)}(x)& {}+{} \mathcal{O}(h^6)& \quad(4) \\
f(x - 2h) &= f(x) {}-{} &2hf'(x)& {}+{} & \frac{(-2h)^2}{2!}f''(x)& {}+{} & \frac{(-2h)^3}{3!}f'''(x)& {}+{} & \frac{(-2h)^4}{4!}f^{(4)}(x)& {}+{} & \frac{(-2h)^4}{5!}f^{(5)}(x)& {}+{} \mathcal{O}(h^6)& \quad(5)
\end{alignat*}

16 × ((2) + (4)) − ((1) + (5)) − 30 × (3) gives

$$
- f(x + 2h) + 16 f(x + h) - 30 f(x) + 16(f - h) - f(x - 2h) = 12 h^2 f''(x) + \mathcal{O}(h^6)
$$

Therefore

$$
f''(x) = \frac{1}{h^2}\left(-\frac{1}{12}f(x+2h) + \frac{4}{3}f(x+h) - \frac{5}{2}f(x) + \frac{4}{3}f(x-h) - \frac{1}{12}f(x-2h)\right) + \mathcal{O}(h^4)
$$

In [ ]:
@dataclass
class System1D:
    """
    Dataset to solve the Schrödinger equation.

    Notes
    -----
    All the quantities are in Hartree atomic units (hbar = m_e = e = 1.0).
    """
    xmin: float  # first mesh point
    xmax: float  # last mesh point
    nx: int  # number of mesh points
    mass: float = 1.0  # mass (default: 1.0 for an electron)
    charge: float = -1.0  # charge (default: -1.0 for an electron)
    periodic: bool = False  # whether the system is periodic

    def __post_init__(self):
        # If periodic, `xmax` is excluded from the mesh points.
        endpoint = not self.periodic
        self.xs = np.linspace(self.xmin, self.xmax, self.nx, endpoint=endpoint)
        self.ax = self.xmax - self.xmin
        self.deltax = self.xs[1] - self.xs[0]

Remember that the analytical values are
$$
\epsilon_\nu = \frac{\nu^2 \pi^2 \hbar^2}{2 m L^2} \quad (\nu = 1, 2, 3, \dots).
$$

## PBCs: second-order-derivative matrix

### Implementation (3 pt.)

Write a function that compute the *n* × *n* matrix

$$
\frac{\mathrm{d}^2}{\mathrm{d}x^2}
\rightarrow
\frac{1}{\Delta^2}
\begin{pmatrix}
-2 &  1 &    & & & & \color{red}{1} \\
 1 & -2 &  1\\
   &  1 & -2 &  1\\
   &    & \ddots & \ddots & \ddots \\
   &    &    &  1 & -2 &  1\\
   &    &    &    &  1 & -2 &  1\\
\color{red}{1} &    &    &    &    &  1 & -2\\
\end{pmatrix}
$$

which is the basis of the kinetic-energy matrix with the periodic boundary condition (PBC), in the following two ways.
  - `calc_second_derivative_PBC_for(n, delta)`: With Python `for` loops but without `np.eye`
  - `calc_second_derivative_PBC_eye(n, delta)`: with `np.eye`

### Test (1 pt.)

Check that the two functions give the same results for *n* = 10 and Δ = 1.0 using, e.g.,
[`np.array_equal`](https://numpy.org/doc/stable/reference/generated/numpy.array_equal.html).

### Time measurement (1 pt.)

Check the speed of your implementations by doing, e.g., `%timeit calc_second_derivative_PBC_for(1000, 1.0)`.

## Harmonic oscillator (5 pt)

Consider the following harmonic potential for an electron under the Dirichlet boundary condition of $\psi_{-1} = \psi_{n} = 0$.

$$
V(x) = \frac{1}{2}x^2.
$$

- Plot the potential *V*(*x*) above as a function of *x*. Choose some good range for $x$ by yourself.
- Solve the 1D Schrödinger equation for the potential above.

The analytical eigenvalues of the potential above are given by (in Hartree atomic units)

   $$
   \epsilon_\nu = \nu + \frac{1}{2} \quad (\nu = 0, 1, 2, \dots).
   $$

- Compare your numerical results with the analytical ones.

## Sinusoidal potential with PBCs (5 pt)

Consider now the sinusoidal potential given by

$$
V(x) = 256 \sin (2 \pi x),
$$

which has the periodicity of 1 bohr.

- Calculate the kinetic-energy-density matrix **under the periodic boundary conditions (PBCs)**.
- Plot the potential *V*(*x*) as a function of *x*.
- Solve the eigenproblem.
- Plot the wavefunctions and the probability densities of the eigenstates.